In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

history_3a = pd.read_pickle("Case_3/checkpoints/model_ep_40/Run_2/trial_0/train_history.pkl")
history_3b = pd.read_pickle("Case_3/checkpoints/model_ep_40/Run_3/trial_0/train_history.pkl")
history_3c_40 = pd.read_pickle("Case_3/checkpoints/model_ep_40/Run_5/trial_0/train_history.pkl")
history_3c_160 = pd.read_pickle("Case_3/checkpoints/model_ep_160/Run_0/trial_1/train_history.pkl")
history_3acheat = pd.read_pickle("Case_3/checkpoints/model_ep_40/Run_4/trial_0/train_history.pkl")
history_3d = pd.read_pickle("Case_3/checkpoints/model_ep_160/Run_1/trial_0/train_history.pkl")
history_3d_not_balanced = pd.read_pickle("Case_3/checkpoints/model_ep_160/Run_3/trial_0/train_history.pkl")
history_3d_not_balanced_init = pd.read_pickle("Case_3/checkpoints/model_ep_160/Run_11/trial_0/train_history.pkl")
history_3d_cheat = pd.read_pickle("Case_3/checkpoints/model_ep_40/Run_6/trial_0/train_history.pkl")
history_3d_320= pd.read_pickle("Case_3/checkpoints/model_ep_320/Run_0/trial_0/train_history.pkl")
history_3d_subgroup= pd.read_pickle("Case_3/checkpoints/model_ep_160/Run_2/trial_0/train_history.pkl")
history_3e_samples_group= pd.read_pickle("Case_3/checkpoints/model_ep_160/Run_5/trial_0/train_history.pkl")
history_3e_samples_subgroup= pd.read_pickle("Case_3/checkpoints/model_ep_160/Run_6/trial_0/train_history.pkl")

In [ ]:
history_3a[[c for c in history_3a.columns if "cluster" in c]]

In [ ]:
history_3b[[c for c in history_3b.columns if "weight" in c]]

In [ ]:
history_3c[[c for c in history_3c.columns if "cluster" in c]]

In [ ]:
history_3c_160[[c for c in history_3c_160.columns if "cluster" in c]]

In [ ]:
history_3cheat[[c for c in history_3cheat.columns if "cluster" in c]]

In [ ]:
history_3d[[c for c in history_3d.columns if "cluster" in c]]

In [ ]:
history_3d_cheat[[c for c in history_3d_cheat.columns if "cluster" in c]]

In [ ]:
history_3d_320[[c for c in history_3d_320.columns if "cluster" in c]][-200:-150]

In [ ]:
history_3d_subgroup[[c for c in history_3d_subgroup.columns if "cluster" in c and "grad" not in c]]

In [ ]:
history_3e_samples_group[[c for c in history_3e_samples_group.columns if "cluster" in c and "grad" not in c]]

In [ ]:
history_3e_samples_subgroup[[c for c in history_3e_samples_subgroup.columns if "cluster" in c and "grad" not in c]]

In [ ]:
history_3d_not_balanced[[c for c in history_3d_not_balanced.columns if "cluster" in c and "grad" not in c]].iloc[20:70]

In [ ]:
history_3d_not_balanced_init[[c for c in history_3d_not_balanced_init.columns if "cluster" in c and "grad" not in c]][:100]

### Model weights

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport logistic_regression_model
import torch, pandas as pd, os

path = os.getcwd() + "/base_rate_models/checkpoints/model_ep_160/Run_0/trial_0"
combined = path + "/combined/checkpoint.pt"
min = path + "/min/checkpoint.pt"
maj = path + "/maj/checkpoint.pt"

columns = pd.read_csv("../Datasets/adult_dataset/processed_adult.csv").columns.drop("income")
device = torch.device("cpu")

comb_model = logistic_regression_model.Predictor(len(columns)).to(device)
checkpoint = torch.load(combined, map_location=device)
comb_model.load_state_dict(checkpoint['model_state_dict'])

min_model = logistic_regression_model.Predictor(112).to(device)
checkpoint = torch.load(min, map_location=device)
min_model.load_state_dict(checkpoint['model_state_dict'])

maj_model = logistic_regression_model.Predictor(112).to(device)
checkpoint = torch.load(maj, map_location=device)
maj_model.load_state_dict(checkpoint['model_state_dict'])

path = os.getcwd() + "/Case_1/checkpoints/model_ep_40/Run_1/trial_0/checkpoint.pt"
model1 = logistic_regression_model.Predictor(len(columns)).to(device)
checkpoint = torch.load(path, map_location=device)
model1.load_state_dict(checkpoint['model_state_dict'])

m_f_1 =logistic_regression_model.feature_importance(maj_model, columns, k=0)
m_f_2 = logistic_regression_model.feature_importance(min_model, columns, k=0)
m_f_3 = logistic_regression_model.feature_importance(comb_model, columns, k=0)
model1_f =logistic_regression_model.feature_importance(model1, columns, k=0)

In [ ]:
m_f_1

In [ ]:
m_f_2

In [ ]:
m_f_3

In [ ]:
model1_f

In [ ]:
pd.read_pickle("base_rate_models/checkpoints/model_ep_160/Run_0/trial_0/combined/train_history.pkl")

In [ ]:
import getopt
import sys
import os
from fairness_metrics import *
from load_dataset import *
from logistic_regression_model import *

LABEL_COL, PROTECT_COLS, NUM_EPOCH, ID, NUM_TRIALS, NUM_PROXIES, FILE_PATH, VERBOSE, \
LR_RATE, BATCH_SIZE, BALANCE, KEEP, FILTER_MAJ, FILTER_MIN = "income", [
    "gender"], 160, -1, 1, 0, "../Datasets/adult_dataset/processed_adult.csv", 1, 0.001, \
                                     1000, 1, 1, 0, 0

train_test_datasets = load_split_dataset(FILE_PATH, LABEL_COL,
                                         PROTECT_COLS[0],
                                         is_scaled=True,
                                         num_proxy_to_remove=NUM_PROXIES,
                                         balanced=BALANCE,
                                         keep=KEEP,
                                         verbose=VERBOSE,
                                         filters = [FILTER_MAJ, FILTER_MIN])

tr_maj_d, tr_min_d, tr_d, te_maj_d, te_min_d, te_d = train_test_datasets

print("---------- MAPPING ----------")
print("Train: ", tr_d.mapping)
print("Test: ", te_d.mapping)
print("-----------------------------")

device = torch.device("cpu")
num_predictor_features = tr_maj_d[0][0].shape[0]

# Training data loaders
train_maj_loader = torch.utils.data.DataLoader(dataset=tr_maj_d, batch_size=BATCH_SIZE, shuffle=True)
train_min_loader = torch.utils.data.DataLoader(dataset=tr_min_d, batch_size=BATCH_SIZE, shuffle=True)
train_loader = torch.utils.data.DataLoader(dataset=tr_d, batch_size=BATCH_SIZE, shuffle=True)

test_maj_loader = torch.utils.data.DataLoader(dataset=te_maj_d, batch_size=BATCH_SIZE, shuffle=False)
test_min_loader = torch.utils.data.DataLoader(dataset=te_min_d, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=te_d, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model2_pred = Predictor(len(columns)).to(device)
optimizer = optim.Adam(model2_pred.parameters(), lr=LR_RATE)


history_model2 = train(model2_pred, device, train_loader, optimizer, 40, verbose=VERBOSE, minority_w=None)

In [ ]:
train_features, train_labels, train_protect = tr_d.features, tr_d.label, tr_d.protect
test_features, test_labels, test_protect = te_d.features, te_d.label, te_d.protect

hyperparameters = {'adversary_loss_weight': 0.1,
                   'batch_size': BATCH_SIZE,
                   'num_epochs': NUM_EPOCH,
                   }

with tf.Session() as sess:
    model = adversial_model.AdversarialLogisticModel("training", sess, hyperparameters, debias=True)
    trained_model = model.fit(train_features, train_labels, train_protect)
    train_pred_labels = trained_model.predict(train_features, train_labels, train_protect)
    test_pred_labels = trained_model.predict(test_features, test_labels, test_protect)

In [ ]:
accs = equalizing_odds(test_pred_labels, test_labels, test_protect)
accs

In [ ]:
np.average([np.max(acc) - np.min(acc) for acc in accs])

## Model 1

In [ ]:
test_pred_labels, test_loss, test_accuracy, _ = test(model1, device, test_loader)
accs = equalizing_odds(test_pred_labels, test_loader.dataset.label, test_loader.dataset.protect)
accs

In [ ]:
np.average([np.max(acc) - np.min(acc) for acc in accs])

In [ ]:
scatter_plots(device, maj_model, min_model, model1, test_maj_loader, test_min_loader, test_loader)

## Model 2

In [ ]:
test_pred_labels, test_loss, test_accuracy, _ = test(model2_pred, device, test_loader)
accs = equalizing_odds(test_pred_labels, test_loader.dataset.label, test_loader.dataset.protect)
accs

In [ ]:
np.average([np.max(acc) - np.min(acc) for acc in accs])

In [ ]:
scatter_plots(device, maj_model, min_model, model2_pred, test_maj_loader, test_min_loader, test_loader)

## Case 3e

In [ ]:
test_pred_labels, test_loss, test_accuracy, _ = test(comb_model, device, test_loader)
accs = equalizing_odds(test_pred_labels, test_loader.dataset.label, test_loader.dataset.protect)
accs

In [ ]:
np.average([np.max(acc) - np.min(acc) for acc in accs])

In [ ]:
scatter_plots(device, maj_model, min_model, comb_model, test_maj_loader, test_min_loader, test_loader)

In [ ]:
vector_1 = np.array([0,1,1,0,1,0])
vector_2 = np.array([1,1,1,0,1,1])

In [ ]:
np.sum(vector_1 == vector_2)/len(vector_1)